In [1]:
###################################################################################################
# Define parameters and query data
###################################################################################################

# Use this to read in packages from other directories ---------
import sys, os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))) # two dirs back
#--------------------------------------------------------------
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stndata as stndata

variable = 'pcpn' # Rain (Daily Rainfall)

# Location and bounding box
location_name = 'St. Louis, MO'
nlat = 38.9     # northern latitude
slat = 38.5     # southern latitude
wlon = -90.55   # western longitude
elon = -90.17   # eastern longitude

# Query variable
var, meta = stndata.bbox_multistn_daily(elem=variable,slat=slat,nlat=nlat,wlon=wlon,elon=elon,
                                        print_md=False)

pcpn: Reading in 58 total stations (station id: name, state) ...
#1. 112614: EAST ST LOUIS PARKS COLLEGE, IL
#2. 111160: CAHOKIA, IL
#3. 238791: WEBSTER GROVES, MO
#4. 238561: VALLEY PARK, MO
#5. 237452: ST LOUIS SCIENCE CENTER, MO
#6. 93963: ST LOUIS EADS BRIDGE, MO
#7. 237465: ST LOUIS ST LOUIS UNIV, MO
#8. 237470: SAINT LOUIS WASHINGTON UNIV, MO
#9. 238525: UNIVERSITY CITY, MO
#10. US1MOSLC007: ST. LOUIS 5.7 SW, MO
#11. 234272: JEFFERSON BARRACKS, MO
#12. US1MOSL0003: WEBSTER GROVES 0.9 ESE, MO
#13. US1MOSL0004: KIRKWOOD 1.6 S, MO
#14. US1MOSL0018: MANCHESTER 1.4 SE, MO
#15. US1MOSL0020: LADUE 1.6 N, MO
#16. US1MOSLC004: ST. LOUIS 1.5 S, MO
#17. US1MOSLC005: ST. LOUIS 2.4 S, MO
#18. US1MOSL0029: AFFTON 0.8 WNW, MO
#19. US1MOSL0035: ST. LOUIS 1.0 SW, MO
#20. US1MOSLC006: ST. LOUIS 4.9 SW, MO
#21. US1MOSL0045: CRESTWOOD 0.4 NW, MO
#22. US1MOSL0048: ST. LOUIS 6.3 SW (CLOSED), MO
#23. US1MOSL0049: BALLWIN 1.6 E, MO
#24. US1MOSL0050: WEBSTER GROVES 1.6 NNE, MO
#25. US1MOSL0054: WEBSTER G

In [2]:
# PLOT

# Use this to read in packages from other directories ---------
import sys, os 
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))) # two dirs back
#--------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy, cartopy.mpl.geoaxes, cartopy.io.img_tiles
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stnmeta as stnmeta
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stndata as stndata
from ClimateDataVisualizer.processing.bbox_dy import bbox_avg_dy
from ClimateDataVisualizer.processing.bbox_my import bbox_avg_my, bbox_max_my, bbox_min_my
from ClimateDataVisualizer.inset_axes.inset_axes import inset_map, inset_timeseries
import warnings

def cumulative_pcpn_plot(var,meta,location_name,nlat,slat,wlon,elon,syr,eyr,stats,na_allwd,iyr,
             minbuff,maxbuff,majtick,mintick,incl_hist,incl_year,incl_map,img_tile,lbl_buff,ext_buff,
             mltyr=''):

    ###################################################################################################
    # Average data by day and year 
    ###################################################################################################

    # List of leap years from 1800 to 2100
    leapyears = [1800 + i * 4 for i in range((2100 - 1800) // 4 + 1)]
    isleap = False if iyr not in leapyears else True

    # Average by day and year
    var_dy = bbox_avg_dy(var,leap=isleap)

    # Cumulative sum of rainfall, fill all NaNs (except this year) with zeros
    var_cs = var_dy.apply(lambda col: col.fillna(0) if col.name != str(pd.Timestamp.today().year) else col)
    var_cs = var_cs.cumsum()
    var_cs['month'] = var_dy['month']
    var_cs['day']   = var_dy['day']

    # Make year all NaNs if it has more than the threshold - i.e., it won't be shown
    var_cs[var_cs.isna().sum()[var_cs.isna().sum() > na_allwd].iloc[:-1].index] = float('nan')

    ###################################################################################################
    # Auto-select dates based on parameters 
    ###################################################################################################

    # If earliest, take earliest year with enough data (applying allowable missing value limit)
    if syr == 'earliest':
        for c in range(2,len(var_dy)+1):
            if var_dy[var_dy.columns[c]].isna().sum() < na_allwd:
                syr = int(var_dy.columns[c])
                break

    # If latest, take latest year (but not this year) (applying allowable missing value limit)
    if eyr == 'latest':
        for c in range(2,len(var_dy)+1):
            if var_dy[var_dy.columns[-c]].isna().sum() < 365:
                eyr = int(var_dy.columns[-c])
                break

    # Define back-end plotting year based on whether current year is a leap year or not
    plt_yr = 2020 if iyr in leapyears else 2022    

    ###################################################################################################
    # Define figure 
    ###################################################################################################

    fig, ax = plt.subplots(figsize=[8,5],dpi=300)

    # Define x-axis time interval
    xtime = pd.date_range(start=str(plt_yr)+'-01-01', end=str(plt_yr)+'-12-31', freq='D') 

    ############################################################################################################# 
    # PLOT HISTORICAL DATA 
    ############################################################################################################# 

    if incl_hist == True:

        # Choose colormap
        colortable = 'coolwarm'
        colors = plt.get_cmap(colortable)(mpl.colors.Normalize()(range(syr,eyr+1)))

        # Plot each year's cumulative values as a different color
        for i, yr in enumerate(range(syr,eyr+1)):
            ax.plot(xtime,var_cs[str(yr)],'-',c=colors[i],lw=1,alpha=0.5,zorder=0.1)

        # Color bar
        sm = plt.cm.ScalarMappable(cmap=colortable, norm=mpl.colors.Normalize(vmin=syr,vmax=eyr))
        cbar = plt.colorbar(sm,ax=ax,orientation='horizontal',shrink=0.38,anchor=(0.28,7.66))
        cbar.set_ticks(range(syr, eyr+1))
        cbar.set_ticklabels(range(syr, eyr+1),fontsize=8)
        cbar.ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(20))
        minor_multlocator = 5 if eyr-syr > 40 else 1 # if period is <40, higher minor tick resolution
        cbar.ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(minor_multlocator))
        ax.text(0.17,0.93,r' $\bf{HISTORICAL \ DATA \ PERIOD :}$'+' {}-{}'.format(syr,eyr),
                fontsize=7,transform=ax.transAxes)

        # Plot stats of cumulative values
        if stats == 'Mean':
           var_stat = np.nanmean(var_cs.iloc[:,var_cs.columns.get_loc(str(syr)):\
                                               var_cs.columns.get_loc(str(eyr))+1],axis=1)
           stats_text = 'MEAN'
        if stats == 'Median':
           var_stat = np.nanmedian(var_cs.iloc[:,var_cs.columns.get_loc(str(syr)):\
                                                 var_cs.columns.get_loc(str(eyr))+1],axis=1)
           stats_text = 'MEDIAN'
        ax.plot(xtime,var_stat,'--',c='k',alpha=0.5,lw=1.5,zorder=10)
        ax.text(pd.to_datetime(str(plt_yr)+'-12-31'),var_stat[-1]+1,stats_text,ha='right',fontsize=8,
                weight='bold',alpha=0.75)

    ############################################################################################################# 
    # PLOT INDIVIDUAL YEAR 
    ############################################################################################################# 

    if incl_year == True:
        ax.plot(xtime,var_cs[str(iyr)],'-',c='k',lw=2,zorder=1000)

    if mltyr == '':
        # Make legend for iyr
        leg_year = ax.legend([mpl.lines.Line2D([0],[0],c='k',linestyle='-',lw=2)],
                             (str(iyr),''),framealpha=0.,loc='upper left',
                             prop={'weight': 'bold', 'size': 8})
        leg_year.set_bbox_to_anchor((0.16,0.77))
    else:
        # Add in-graph text for iyr
        ax.text(xtime[np.count_nonzero(~np.isnan(var_cs[str(iyr)]))]+pd.Timedelta(days=3),
                var_cs[str(iyr)].iloc[np.count_nonzero(~np.isnan(var_cs[str(iyr)]))-1],
                r'$\bf{'+str(iyr)+'}$',color='k',fontsize=9,zorder=1000)
        
        # Plot each additional year
        mltyr = mltyr.split(',')
        tab10 = [mpl.colormaps['tab10'](i) for i in range(mpl.colormaps['tab10'].N)]
        mltyr_col = [tab10[3],tab10[1],tab10[2],tab10[0]]+tab10[4:]
        for i in range(len(mltyr)):
            ax.plot(xtime,var_cs[mltyr[i]],'-',c=mltyr_col[i],lw=2,zorder=999)
            ax.text(pd.to_datetime(str(plt_yr)+'-12-31')+pd.Timedelta(days=3),var_cs[mltyr[i]].iloc[-1],
                    r'$\bf{'+str(mltyr[i])+'}$',color=mltyr_col[i],fontsize=8,zorder=999)

    ############################################################################################################# 
    # Features to include? Can toggle on and off in parameters
    ############################################################################################################# 
        
    if incl_map == True:
        inset_map(ax=ax,meta=meta,var=var,width=0.7,height=0.8,markercolor='k',incl_year=incl_year,iyr=iyr,
                  iyr_col='tab:red',slat=slat,nlat=nlat,wlon=wlon,elon=elon,bbox_to_anchor=(0,0,0.142,0.95),
                  lbl_buff=lbl_buff,proj=cartopy.crs.PlateCarree(),ext_buff=ext_buff,img_tile=img_tile)

    ############################################################################################################# 
    # Title and text
    ############################################################################################################# 

    ax.text(1.,0.025,r'$\bf{DATA:}$'+' NOAA ACIS (http://data.rcc-acis.org)'+\
                     r'$\ \ \bf{IMAGE:}$ Alex Thompson (@ajtclimate)',
                     ha='right',va='center',fontsize=5,transform=ax.transAxes);
    ax.set_title('Cumulative Rainfall in '+location_name,loc='center',fontsize=14,pad=5,
                 weight='bold');

    ############################################################################################################# 
    # Axes specs 
    ############################################################################################################# 

    ax.set_xticks([str(plt_yr)+'-'+'{:02}'.format(month) for month in range(1, 13)])
    ax.set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
    ax.set_ylabel('Region Mean (inches)')
    ax.get_yaxis().set_major_locator(mpl.ticker.MultipleLocator(majtick))
    ax.get_yaxis().set_minor_locator(mpl.ticker.MultipleLocator(mintick))
    ax.grid(alpha=0.05)
    ax.set_xlim([pd.to_datetime(str(plt_yr-1)+'-12-30'),pd.to_datetime(str(plt_yr+1)+'-01-01')])
    ax.set_ylim([0-np.nanmax(var_cs.iloc[2:])*0.02-minbuff,np.nanmax(var_cs.iloc[2:])+maxbuff]);
    ax.spines[['right','top']].set_visible(False)

    return fig, var_cs


In [3]:
# WIDGET

# Use this to read in packages from other directories ---------
import sys, os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))) # two dirs back
#--------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy, cartopy.mpl.geoaxes, cartopy.io.img_tiles
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stnmeta as stnmeta
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stndata as stndata
from ClimateDataVisualizer.processing.bbox_dy import bbox_avg_dy
from ClimateDataVisualizer.inset_axes.inset_axes import inset_map, inset_timeseries
from ClimateDataVisualizer.interactives import plots
from ClimateDataVisualizer.downloads.file_options import pdf_opts, xcl_opts         
import ipywidgets as ipyw
from IPython.display import display, HTML

# URLs to use for user help guides, needs double quotes
url_annualcycle = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-annual-cycle"
url_timeseries  = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-time-series"
url_cumulative  = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-cumulative"
url_map         = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-map-of-stations"
url_yaxis       = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-y-axis"

def cumulative_pcpn_widget(var,meta,location_name,nlat,slat,wlon,elon):

    #----------------------------------------------------------------------------------------------
    # Set up parameters to toggle
    #----------------------------------------------------------------------------------------------

    # Define years for historical range
    txt_hist = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_cumulative} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Define years for historical range</a></h3>')
    text_hist = ipyw.Label(value='Include historical climate?',layout=ipyw.Layout(width='160px'))
    incl_hist = ipyw.Dropdown(options=[True,False],value=True,layout=ipyw.Layout(width='60px'))
    text_syr = ipyw.Label(value='Starting Year',layout=ipyw.Layout(width='120px'))
    text_eyr = ipyw.Label(value='Ending Year',layout=ipyw.Layout(width='120px'))
    syr = ipyw.Dropdown(options=['earliest',*np.sort(var['Date'].dt.year.unique())[:-1]],
                        value='earliest',layout=ipyw.Layout(width='100px'))
    eyr = ipyw.Dropdown(options=['latest',*np.sort(var['Date'].dt.year.unique())[::-1][1:]],
                        value='latest',layout=ipyw.Layout(width='100px'))
    text_stats = ipyw.Label(value='Display statistics',layout=ipyw.Layout(width='120px'))
    stats = ipyw.Dropdown(options=['Mean','Median'],value='Mean',layout=ipyw.Layout(width='100px'))

    # Plot individual year
    txt_indv = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_cumulative} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Plot individual year</a></h3>')
    text_year = ipyw.Label(value='Include individual year?',layout=ipyw.Layout(width='160px'))
    incl_year = ipyw.Dropdown(options=[True,False],value=True,layout=ipyw.Layout(width='70px'))
    text_iyr = ipyw.Label(value='Year Displayed',layout=ipyw.Layout(width='90px'))
    iyr = ipyw.Dropdown(options=var['Date'].dt.year.unique()[::-1],
                        value=var['Date'].dt.year.unique()[-1],layout=ipyw.Layout(width='70px'))
    text_mltyr = ipyw.Label(value='Add Years',layout=ipyw.Layout(width='65px'))
    mltyr = ipyw.Text(placeholder='e.g., 2022,2021 (max 9)',layout=ipyw.Layout(width='165px'),
                      value='',continuous_update=False)
    
    # Missing data allowed?
    txt_miss = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_cumulative} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Missing data allowed</a></h3>')
    text_md  = ipyw.Label(value='missing days per year',layout=ipyw.Layout(width='135px'))
    na_allwd = ipyw.Dropdown(options=list(np.arange(1,367)),value=100,
                             layout=ipyw.Layout(width='80px'))

    # Y-axis buffer
    txt_buff  = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_yaxis} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Y-axis buffer</a></h3>')
    maxb_desc = ipyw.Label(value='Buffer above y-axis',layout=ipyw.Layout(width='120px'))
    maxbuff   = ipyw.FloatText(value=1.,layout=ipyw.Layout(width='60px'))
    maxb_unit = ipyw.Label(value='in')
    minb_desc = ipyw.Label(value='Buffer below y-axis',layout=ipyw.Layout(width='120px'))
    minbuff   = ipyw.FloatText(value=0.,layout=ipyw.Layout(width='60px'))
    minb_unit = ipyw.Label(value='in')

    # Y-axis tick stride
    txt_tick   = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_yaxis} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Y-axis tick stride</a></h3>')
    majtk_desc = ipyw.Label(value='Major tick stride',layout=ipyw.Layout(width='120px'))
    majtick    = ipyw.FloatText(value=10.,layout=ipyw.Layout(width='60px'))
    majtk_unit = ipyw.Label(value='in')
    mintk_desc = ipyw.Label(value='Minor tick stride',layout=ipyw.Layout(width='120px'))
    mintick    = ipyw.FloatText(value=1.,layout=ipyw.Layout(width='60px'))
    mintk_unit = ipyw.Label(value='in')

    # Map
    txt_map   = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_map} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Map of stations</a></h3>')
    text_map  = ipyw.Label(value='Include map?',layout=ipyw.Layout(width='100px'))
    incl_map  = ipyw.Dropdown(options=[True,False],value=True,layout=ipyw.Layout(width='80px'))
    text_lyr  = ipyw.Label(value='Image tile',layout=ipyw.Layout(width='100px'))
    img_tile  = ipyw.Dropdown(options=['QuadtreeTiles','GoogleTiles','OpenStreetMap','grey'],
                              value='QuadtreeTiles',layout=ipyw.Layout(width='80px'))
    text_lbl = ipyw.Label(value='Label distance',layout=ipyw.Layout(width='100px'))
    lbl_buff = ipyw.Dropdown(options=[('10%',0.1),('20%',0.2),('30%',0.3),('40%',0.4),('50%',0.5),
                                       ('60%',0.6),('70%',0.7),('80%',0.8),('90%',0.9),('None',1.5)],
                              value=0.3,layout=ipyw.Layout(width='80px'))
    text_ext = ipyw.Label(value='Lat/lon buffer',layout=ipyw.Layout(width='100px'))
    ext_buff = ipyw.Dropdown(options=[('0.1°',0.1),('0.25°',0.25),('0.5°',0.5),('1°',1.),('5°',5.)],
                              value=0.5,layout=ipyw.Layout(width='80px'))
    
    #----------------------------------------------------------------------------------------------
    # Layout for dropdowns
    #----------------------------------------------------------------------------------------------

    ui = ipyw.VBox([
                    # FIRST ROW
                    ipyw.HBox([ipyw.VBox([
                                 # Define years for historical range
                                 ipyw.HBox([txt_hist],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([text_hist,incl_hist],layout=ipyw.Layout(
                                                                        justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([text_syr,syr]),
                                                       ipyw.HBox([text_eyr,eyr])])],
                                           layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([text_stats,stats],
                                           layout=ipyw.Layout(justify_content='center'))]),
                               ipyw.VBox([
                                 # Plot individual year
                                 ipyw.HBox([txt_indv],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([text_year,incl_year],
                                           layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([text_iyr,iyr],
                                           layout=ipyw.Layout(justify_content='space-between')),
                                 ipyw.HBox([text_mltyr,mltyr])]),
                               ipyw.VBox([
                                 # Map 
                                 ipyw.HBox([txt_map],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([text_map,incl_map]),
                                                       ipyw.HBox([text_lyr,img_tile]),
                                                       ipyw.HBox([text_lbl,lbl_buff]),
                                                       ipyw.HBox([text_ext,ext_buff])])])])],
                               layout=ipyw.Layout(justify_content='space-around')),
                    # SECOND ROW
                    ipyw.HBox([ipyw.VBox([
                                 # Missing data allowed per year
                                 ipyw.HBox([txt_miss],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([na_allwd,text_md],
                                           layout=ipyw.Layout(justify_content='center'))]),
                               ipyw.VBox([
                                 # Y-axis buffer
                                 ipyw.HBox([txt_buff],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([maxb_desc,maxbuff,maxb_unit]),
                                                       ipyw.HBox([minb_desc,minbuff,minb_unit])])],
                                           layout=ipyw.Layout(flex='1 1 auto',
                                                              justify_content='center'))]),
                               ipyw.VBox([
                                 # Y-axis tick stride
                                 ipyw.HBox([txt_tick],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([majtk_desc,majtick,majtk_unit]),
                                                       ipyw.HBox([mintk_desc,mintick,mintk_unit])])],
                                            layout=ipyw.Layout(flex='1 1 auto',
                                                               justify_content='space-between'))])],
                               layout=ipyw.Layout(justify_content='space-around'))
                     ])

    #----------------------------------------------------------------------------------------------
    # Display interactive plot
    #----------------------------------------------------------------------------------------------

    # Define function with only interactive components that runs widget.plot function
    def plot_fig(syr,eyr,stats,na_allwd,iyr,mltyr,minbuff,maxbuff,majtick,mintick,incl_hist,incl_year,
                 incl_map,img_tile,lbl_buff,ext_buff):

        fig,var_cs = plots.cumulative_pcpn_plot(
                     var=var,meta=meta,na_allwd=na_allwd,location_name=location_name,
                     nlat=float(nlat),slat=float(slat),wlon=float(wlon),elon=float(elon),
                     syr=syr,eyr=eyr,stats=stats,iyr=iyr,mltyr=mltyr,minbuff=minbuff,maxbuff=maxbuff,
                     majtick=majtick,mintick=mintick,incl_hist=incl_hist,incl_year=incl_year,
                     incl_map=incl_map,img_tile=img_tile,lbl_buff=lbl_buff,ext_buff=ext_buff)

        # Create a button widget to download the figure as PDF
        pdf_output = ipyw.Output()
        def download_pdf(button):
           pdf_filename = 'figure.pdf'
           pdf_opts(fig=fig,pdf_output=pdf_output,pdf_filename=pdf_filename)
        pdf_but = ipyw.Button(description='Download Figure', layout={'width': '140px'})
        pdf_but.on_click(download_pdf)

        # Create a button widget to download data as Excel
        xcl_output = ipyw.Output()
        def download_xcl(button):
            with xcl_output: print('Download initiated. Can take more than a minute if over 100 stations '+
                                   'are queried. Do not click download button again.')
            xcl_filename = 'data.xlsx'
            with pd.ExcelWriter(xcl_filename) as w:
                var.assign(**{var.columns[0]: var.iloc[:,0].astype(str)}
                           ).to_excel(w,sheet_name='stations',index=False)
                meta.to_excel(w,sheet_name='metadata',index=False)
                if incl_hist == True:
                    pd.concat([pd.DataFrame({'month':var_cs['month'],'day':var_cs['day']}),
                               var_cs.iloc[:,2:]],axis=1).to_excel(w,sheet_name='historical',index=False)
                if incl_year == True:
                    pd.DataFrame({'month':var_cs['month'],'day':var_cs['day'],str(iyr):var_cs[str(iyr)]}
                                ).to_excel(w,sheet_name=str(iyr),index=False)
            xcl_opts(xcl_filename=xcl_filename,xcl_output=xcl_output)
        xcl_but = ipyw.Button(description='Download Data',layout=ipyw.Layout(width='140px'))
        xcl_but.on_click(download_xcl)

        # Display download buttons together
        button_box = ipyw.VBox([ipyw.HBox([pdf_but,xcl_but],layout=ipyw.Layout(justify_content='center'))])
        display(button_box,pdf_output,xcl_output)

    # Interactive output with only interactive components in dictionary
    out = ipyw.interactive_output(plot_fig,{'syr':syr,'eyr':eyr,'stats':stats,'na_allwd':na_allwd,
                                            'iyr':iyr,'mltyr':mltyr,'minbuff':minbuff,'maxbuff':maxbuff,
                                            'majtick':majtick,'mintick':mintick,
                                            'incl_hist':incl_hist,'incl_year':incl_year,
                                            'incl_map':incl_map,'img_tile':img_tile,
                                            'lbl_buff':lbl_buff,'ext_buff':ext_buff})
    display(ui,out)


In [4]:
#-----------------
# Run function    
#-----------------

cumulative_pcpn_widget(var=var,meta=meta,location_name=location_name,nlat=nlat,slat=slat,wlon=wlon,elon=elon)


Output()